# Download Hindi resources for NLTK

In [1]:
import nltk
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Emoji pattern and other special character removal


In [2]:
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# Download Hindi resources for NLTK
nltk.download('punkt')

# Emoji pattern for removal
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642"
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
    "]+", flags=re.UNICODE)

# Hindi stopwords list (expanded)
HINDI_STOPWORDS = {
    'अंदर', 'अत', 'अदि', 'अप', 'अपना', 'अपनि', 'अपनी', 'अपने', 'अभि', 'अभी', 'आदि',
    # ... (rest of your stopwords remain the same)
    'होते', 'होना', 'होने'
}



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **Data Preprocessing**

In [3]:
def preprocess_text(text):
    """Clean and normalize Hindi text"""
    # Remove emojis
    text = emoji_pattern.sub(r'', str(text))
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters except Hindi punctuation
    text = re.sub(r'[^\w\u0900-\u097F।?!,.]+', ' ', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# **Custom Hindi sentence tokenizer**

In [4]:
def hindi_sentence_tokenize(text):

    sentences = []
    current = []
    for char in text:
        current.append(char)
        if char in {'।', '?', '!'}:
            sentences.append(''.join(current).strip())
            current = []
    if current:
        sentences.append(''.join(current).strip())
    return [s for s in sentences if s]

#**Tokenization - Sentence Tokenization and Word**
#**TF-IDF Calculation, Sentence Scoring.**

In [5]:
def hindi_word_tokenize(text):
    return re.findall(r"[\w\u0900-\u097F]+", text)

def hindi_text_summarizer(text):
    # Step 1: Preprocess and clean text
    cleaned_text = preprocess_text(text)

    # Step 2: Sentence Tokenization
    sentences = hindi_sentence_tokenize(cleaned_text)

    # Step 3: Word Tokenization and Text Cleaning
    preprocessed_sentences = []
    for sentence in sentences:
        words = hindi_word_tokenize(sentence)
        cleaned_words = [word for word in words
                       if word not in HINDI_STOPWORDS and len(word) > 1]
        preprocessed_sentences.append(' '.join(cleaned_words))

            # Step 4: TF-IDF Calculation
    vectorizer = TfidfVectorizer()
    try:
        tfidf_matrix = vectorizer.fit_transform(preprocessed_sentences)
    except ValueError:
        return "Unable to generate summary (text may be too short or contain no meaningful words)"

    # Step 5: Sentence Scoring
    sentence_scores = {}
    for i, sentence in enumerate(preprocessed_sentences):
        score = sum(
            tfidf_matrix[i, vectorizer.vocabulary_[word]]
            for word in sentence.split()
            if word in vectorizer.vocabulary_
        )
        sentence_scores[i] = score

    # Select important sentences (30-50% of original)
    select_length = max(1, min(len(sentences), len(sentences)//3))
    top_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)[:select_length]

    # Step 6: Generate Summary
    selected_indices = sorted([i for i, _ in top_sentences])
    summary = ' '.join([sentences[i] for i in selected_indices]).strip()

    # Add proper punctuation if missing
    if summary and not any(summary.endswith(p) for p in ['।', '?', '!']):
        summary += '।'

    return summary if summary else "No summary could be generated"



# **Input**

In [ ]:


if __name__ == "__main__":
    print("हिंदी पाठ सारांशकर्ता")
    print("---------------------")
    print("कृपया हिंदी पाठ दर्ज करें (समाप्त करने के लिए केवल 'समाप्त' लिखें):\n")

    while True:
        hindi_text = input()
        if hindi_text.lower().strip() == 'समाप्त':
            break

        summary = hindi_text_summarizer(hindi_text)

        print("\nमूल पाठ लंबाई:", len(hindi_text), "वर्ण")
        print("सारांश लंबाई:", len(summary), "वर्ण")
        print("\nसारांश:")
        print(summary)
        print("\n---------------------")
        print("अगला पाठ दर्ज करें या 'समाप्त' लिखें:")

हिंदी पाठ सारांशकर्ता
---------------------
कृपया हिंदी पाठ दर्ज करें (समाप्त करने के लिए केवल 'समाप्त' लिखें):

आज के व्यस्त जीवन में हर कोई अपनी दिनचर्या में कुछ न कुछ बदलाव चाहता है. सुबह जल्दी उठना और व्यायाम करना स्वस्थ रहने के लिए बहुत ज़रूरी है. एक अच्छी दिनचर्या हमें अनुशासित रहने और समय पर अपने कार्य पूरे करने में मदद करती है. पौष्टिक नाश्ता करना और समय पर स्कूल या काम पर जाना भी महत्वपूर्ण है. शाम को दोस्तों के साथ या परिवार के साथ समय बिताना हमारे मन को शांत करता है.

मूल पाठ लंबाई: 369 वर्ण
सारांश लंबाई: 370 वर्ण

सारांश:
आज के व्यस्त जीवन में हर कोई अपनी दिनचर्या में कुछ न कुछ बदलाव चाहता है. सुबह जल्दी उठना और व्यायाम करना स्वस्थ रहने के लिए बहुत ज़रूरी है. एक अच्छी दिनचर्या हमें अनुशासित रहने और समय पर अपने कार्य पूरे करने में मदद करती है. पौष्टिक नाश्ता करना और समय पर स्कूल या काम पर जाना भी महत्वपूर्ण है. शाम को दोस्तों के साथ या परिवार के साथ समय बिताना हमारे मन को शांत करता है.।

---------------------
अगला पाठ दर्ज करें या 'समाप्त' लिखें:
आज की दुनिया में पर्यावरण सं